In [1]:
#Importing all the packages for the execution of the code
import pandas as pd
import numpy as np
import collections
import itertools

In [2]:
df_balance= pd.read_csv("/content/drive/MyDrive/datasets/balance-scale.csv")
dict_test={}

df_balance=df_balance.replace('R','Positive')
df_balance=df_balance.replace('L','Negative')

df_training=df_balance.sample(frac=0.7)#spliting the data into traning and testing 

df_testing=pd.concat([df_balance, df_training, df_training]).drop_duplicates(keep=False)  
dict_test=df_testing['Class'].to_dict()
dict_uni_values=df_training.T.apply(lambda x: x.nunique(), axis=1).to_dict()#seeing all the unique values





In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
list_uni_values=[]
for i in df_training['Class']:
    list_uni_values.append(i)
unique=set(list_uni_values) 

dict_prior={}
prior = df_training.groupby("Class").size().div(len(df_training))
dict_prior=prior.to_dict()#converting the probability df to dictionary
total_values_inclass={}
total_values_inclass = df_training.groupby("Class").size().to_dict()#calculating the total values in the class


In [5]:
value_count_columns={}
for column in df_training.columns:
    if column != "Class":
        sd=(df_training.groupby(column)["Class"].value_counts()+1).to_dict()
        value_count_columns[column]=sd    

In [6]:
#doing m-estimate, its not required but putting it up
condition_prob={}
condition_prob_col={}
for key,value in value_count_columns.items():
    for key1,value1 in value.items():
        total=(value1/(total_values_inclass[key1[1]]+dict_uni_values[key]))
        condition_prob[key1]=total
    condition_prob_col.setdefault(key,{}).update(condition_prob)
print(condition_prob_col)    
 
        
        

{'LWght': {(1, 'Positive'): 0.68, (1, 'Negative'): 0.5, (2, 'Positive'): 0.32, (2, 'Negative'): 0.5}, 'LDist': {(1, 'Positive'): 0.3106796116504854, (1, 'Negative'): 0.5, (2, 'Positive'): 0.3106796116504854, (2, 'Negative'): 0.18518518518518517, (3, 'Positive'): 0.1553398058252427, (3, 'Negative'): 0.2962962962962963, (4, 'Positive'): 0.0970873786407767, (4, 'Negative'): 0.2222222222222222, (5, 'Positive'): 0.1262135922330097, (5, 'Negative'): 0.25925925925925924}, 'RWght': {(1, 'Positive'): 0.11650485436893204, (1, 'Negative'): 0.48148148148148145, (2, 'Positive'): 0.22330097087378642, (2, 'Negative'): 0.25925925925925924, (3, 'Positive'): 0.22330097087378642, (3, 'Negative'): 0.18518518518518517, (4, 'Positive'): 0.20388349514563106, (4, 'Negative'): 0.2222222222222222, (5, 'Positive'): 0.23300970873786409, (5, 'Negative'): 0.25925925925925924}, 'RDist': {(1, 'Positive'): 0.0970873786407767, (1, 'Negative'): 0.4444444444444444, (2, 'Positive'): 0.18446601941747573, (2, 'Negative'): 0

In [7]:
cloumn_indexing={}
prediction_labelling={}
predic_of_R={}
predic_of_L={}

column_list=["LWght","LDist","RWght","RDist"]

for x in column_list:
    col_dex=[df_testing.columns.get_loc(x)]
for x in column_list:
    cloumn_indexing[df_testing.columns.get_loc(x)]=x

for index,row in df_testing.iterrows():
    R_calculate=1
    L_calculate=1
    for columns in range(1,5):
    
        R_calculate=R_calculate*condition_prob_col[cloumn_indexing[columns]][(row[cloumn_indexing[columns]],'Positive')]
        L_calculate=L_calculate*condition_prob_col[cloumn_indexing[columns]][(row[cloumn_indexing[columns]],'Negative')]   
    
    predic_of_R=total_values_inclass['Positive']*R_calculate
    predic_of_L=total_values_inclass['Negative']*L_calculate
    
    label='Positive' if predic_of_R>predic_of_L else 'Negative'
    prediction_labelling[index]=label
print(prediction_labelling)        



{0: 'Negative', 7: 'Positive', 9: 'Negative', 10: 'Positive', 12: 'Positive', 14: 'Negative', 16: 'Positive', 22: 'Positive', 23: 'Positive', 24: 'Negative', 25: 'Positive', 37: 'Positive', 44: 'Positive', 45: 'Positive', 47: 'Negative', 48: 'Negative', 51: 'Negative', 52: 'Positive', 53: 'Positive', 59: 'Positive', 63: 'Positive', 64: 'Positive', 65: 'Negative', 67: 'Positive', 76: 'Positive', 77: 'Positive', 79: 'Positive', 83: 'Positive', 84: 'Positive', 87: 'Negative', 90: 'Positive', 91: 'Positive', 94: 'Positive', 104: 'Positive', 105: 'Positive', 106: 'Negative', 108: 'Positive', 115: 'Negative', 116: 'Negative', 126: 'Positive', 130: 'Positive', 131: 'Positive', 132: 'Positive', 135: 'Positive', 140: 'Positive', 147: 'Positive', 148: 'Positive', 151: 'Positive', 156: 'Positive', 160: 'Negative', 168: 'Positive', 170: 'Positive'}


In [8]:
df_predicted=pd.DataFrame.from_dict(prediction_labelling,orient='index')
df_predicted.columns=["Predicted Label"]



In [9]:
TPC=0
TNC=0
FPC=0
FNC=0
for key in dict_test.keys():
    for key1 in prediction_labelling.keys():
        if key == key1:
            if dict_test[key] == 'Positive' and prediction_labelling[key] == dict_test[key]:
                TPC+=1
            elif dict_test[key] == 'Negative' and prediction_labelling[key] == dict_test[key]:
                TNC+=1
            elif dict_test[key] == 'Positive' and prediction_labelling[key] != dict_test[key]:
                FPC+=1
            elif dict_test[key] == 'Negative' and prediction_labelling[key] != dict_test[key]:    
                FNC+=1
#print(TPC,TNC,FPC,FNC)                
                
                

In [10]:
Accuracy=(TPC+TNC)/(TPC+TNC+FPC+FNC)
print("Accuracy=",round(Accuracy*100,2),"% ")

Accuracy= 84.62 % 
